# 🌍 AI Blog Writer with Multilingual Support, Style Control, SEO, and Image Suggestions
Final Year Project - Enhanced Version

**Features**:
- Generate blogs using GPT-2
- Translate to multiple languages
- Control tone/style (formal, informal, persuasive)
- SEO keyword suggestions using KeyBERT
- Two image suggestions from Unsplash


In [1]:
!pip install transformers gradio keybert sentence-transformers deep-translator requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [15]:
from transformers import pipeline, set_seed
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from deep_translator import GoogleTranslator
import gradio as gr
import requests
import json

# Load GPT-2 and KeyBERT
generator = pipeline('text-generation', model='gpt2-large')
set_seed(42)
kw_model = KeyBERT(model=SentenceTransformer('all-MiniLM-L6-v2'))


Device set to use cpu


In [16]:
def generate_blog(topic, lang, tone):
    prompt = f"Write a {tone.lower()} blog about: {topic}\n\n"
    output = generator(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']

    # Translate
    if lang != "en":
        translated = GoogleTranslator(source='auto', target=lang).translate(text=output)
    else:
        translated = output

    # Keywords
    keywords = kw_model.extract_keywords(translated, stop_words='english', top_n=5)
    keywords = [kw[0] for kw in keywords]

    return translated, keywords


In [17]:
with gr.Blocks() as demo:
    gr.Markdown("## ✍️ AI Blog Writer")

    with gr.Row():
        topic = gr.Textbox(label="Enter Blog Topic", placeholder="e.g., Green AI in Education")
        lang = gr.Dropdown(label="Select Language", choices=["en", "hi", "fr", "es"], value="en")
        tone = gr.Radio(label="Select Tone/Style", choices=["Formal", "Informal", "Persuasive"], value="Formal")

    generate_btn = gr.Button("Generate Blog")

    blog_output = gr.Textbox(label="Generated Blog")
    seo_keywords = gr.Textbox(label="SEO Keywords")

    generate_btn.click(fn=generate_blog, inputs=[topic, lang, tone], outputs=[blog_output, seo_keywords])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e07354674c154d6372.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
from transformers import pipeline, set_seed
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from deep_translator import GoogleTranslator
import gradio as gr

# Load models
generator = pipeline('text-generation', model='gpt2-large')
set_seed(42)
kw_model = KeyBERT(model=SentenceTransformer('all-MiniLM-L6-v2'))

# Blog generation function
def generate_blog(topic, lang, tone):
    prompt = f"Title: {topic}\n\n"
    prompt += f"This is a {tone.lower()} style blog post discussing {topic.lower()}. Let's dive into an engaging, informative, and SEO-friendly article that provides real value to readers.\n\n"
    prompt += f"Introduction:\n"
    output = generator(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']

    # Translate if necessary
    if lang != "en":
        translated = GoogleTranslator(source='auto', target=lang).translate(text=output)
    else:
        translated = output

    # Extract SEO keywords
    keywords = kw_model.extract_keywords(translated, stop_words='english', top_n=5)
    keyword_list = [kw[0] for kw in keywords]
    keyword_str = ", ".join(keyword_list)

    # Combine output
    final_output = f"{translated}\n\n📌 SEO Keywords: {keyword_str}"
    return final_output

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## ✍️ AI Blog Writer")

    with gr.Row():
        topic = gr.Textbox(label="Enter Blog Topic", placeholder="e.g., Green AI in Education")
        lang = gr.Dropdown(label="Select Language", choices=["en", "hi", "fr", "es"], value="en")
        tone = gr.Radio(label="Select Tone/Style", choices=["Formal", "Informal", "Persuasive"], value="Formal")

    generate_btn = gr.Button("Generate Blog")

    blog_output = gr.Textbox(label="Generated Blog & SEO Keywords", lines=20)

    generate_btn.click(fn=generate_blog, inputs=[topic, lang, tone], outputs=[blog_output])

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e98c72589d87643f9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
